In [1]:
%load_ext autoreload
%autoreload 2
import re
import numpy as np
import tensorflow as tf
import os
import sys
from tf2gpt.model import GPT
from utils.story_util import Story,Stories
from utils.progress_bar import ProgressBar
from tensorboardX import SummaryWriter
from tensorflow.keras.utils import multi_gpu_model
import random

In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
tf.keras.backend.set_floatx('float16')

In [5]:
#mirrored_strategy = tf.distribute.MirroredStrategy()

In [6]:
#with mirrored_strategy.scope():
gpt = GPT(
    vocab_size=30_000,
    layer_size=32,
    block_size=1024,
    embedding_dropout=0.0,
    embedding_size=2560,
    num_attention_heads=32,
    attention_dropout=0.0,
    residual_dropout=0.0,
    train_size=499
)
gpt.load_weights('./gpt_weight_pretrain/weight_fp16')

#input_x = tf.keras.layers.Input((499,), dtype=tf.int32)
#outputs = gpt_origin(input_x)

#gpt = tf.keras.Model(inputs=input_x, outputs=outputs)
#gpt = multi_gpu_model(gpt, gpus=8)

print(tf.keras.backend.floatx(), tf.float16, tf.keras.backend.floatx() == tf.float16)
if tf.keras.backend.floatx() == tf.float16:
    for x in gpt.weights:
        assert x.dtype == tf.float16


gpt.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Optimizer
    # Loss function to minimize
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
)

float16 <dtype: 'float16'> True


In [7]:
from utils.gpt2_tokenizer import GPT2Tokenizer
cbpe = GPT2Tokenizer(
    'CPM-Generate/bpe_3w_new/vocab.json',
    'CPM-Generate/bpe_3w_new/merges.txt',
    model_file='CPM-Generate/bpe_3w_new/chinese_vocab.model')

In [8]:
ids = cbpe.encode('今天天气还行')
ids

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.577 seconds.
Prefix dict has been built successfully.


[837, 259, 497, 57, 8, 237]

In [9]:
def test_basic_logic():
    ids = cbpe.encode('今天天气还行')
    #print(ids)
    print("+" * 20)
    for i in range(10):
        output = gpt(tf.constant([ids]))
        #print(output[0].shape)
        nid = np.argmax(output[0, -1])
        ids += [nid]
        #print(i, cbpe.decode(ids))
        #print(np.argmax(output[0],axis=-1))
        print(cbpe.decode(np.argmax(output[0],axis=-1)))
        print('-' * 30)
test_basic_logic()

++++++++++++++++++++
气 不错 不错暖和 
------------------------------
气 不错 不错暖和 ,
------------------------------
气 不错 不错暖和 , 
------------------------------
气 不错 不错暖和 , 咱
------------------------------
气 不错 不错暖和 , 咱们
------------------------------
气 不错 不错暖和 , 咱们 到
------------------------------
气 不错 不错暖和 , 咱们 到 
------------------------------
气 不错 不错暖和 , 咱们 到 郊
------------------------------
气 不错 不错暖和 , 咱们 到 郊外
------------------------------
气 不错 不错暖和 , 咱们 到 郊外 去
------------------------------


In [12]:
def get_learning_rate(learning_rate=6e-4,
                      warmup_steps=20_0000,
                      decay_steps=200_0000,
                      alpha=0.0):
    def decayed_learning_rate(step=1):
        if step <= warmup_steps:
            mult = step / float(warmup_steps)
        else:
            progress = (step - warmup_steps) / (decay_steps - warmup_steps)
            mult = 0.5 * (1 + math.cos(math.pi * progress))
            mult = max(0.1, mult)
        return learning_rate * mult
    return decayed_learning_rate

In [13]:
#stories = Stories("./labeled_data/advanture_translated/processed_translated_story.txt").stories
stories = Stories("./labeled_data/advanture_translated/processed_translated_story.txt").stories
#stories = stories[:50]
data_folder = "./labeled_data/"
txt_files = [(data_folder + i) for i in os.listdir(data_folder) if "txt" in i]
#stories = stories[:10]
stories = [Story("","").from_file(i) for i in txt_files]

In [14]:
valid_stories = Stories("./labeled_data/advanture_translated/processed_translated_story_valid.txt").stories

In [15]:
len(stories),len(valid_stories)

(4, 35)

In [16]:
import copy
def data_generator(stories, batch_size=4,sample_len=200,inf=False):
    while True:
        batch_data = []
        tmp_stories = copy.copy(stories)
        random.shuffle(tmp_stories)
        for i,one_story in enumerate(tmp_stories):
            story_content = one_story.to_dungeon_format()
            story_content = story_content.replace("<start>\n","")
            story_content = story_content.replace("\n<end>","")
            story_content = story_content.replace("\n<end>","")
            story_content = story_content.replace(" ","")
            ids = cbpe.encode(story_content)
            while ids:
                sample = ids[:sample_len]
                ids = ids[sample_len:]
                if len(sample) < sample_len:
                    sample += [0 for i in range((sample_len - len(sample)))]
                batch_data.append(sample)
                if len(batch_data) >= batch_size:
                    yield np.asarray(batch_data)
                    batch_data = []
        if not inf:
            break
            
import copy
def data_generator_content(texts, batch_size=4,sample_len=200,inf=False):
    tmp_text = copy.copy(texts)
    segments = []
    while tmp_text:
        story_content = tmp_text[:10000]
        segments.append(story_content)
        tmp_text = tmp_text[10000:]
    while True:
        
        batch_data = []
        random.shuffle(segments)
        for one_story_context in segments:
            ids = cbpe.encode(one_story_context)
            while ids:
                sample = ids[:sample_len]
                ids = ids[sample_len:]
                if len(sample) < sample_len:
                    sample += [0 for i in range((sample_len - len(sample)))]
                batch_data.append(sample)
                if len(batch_data) >= batch_size:
                    yield np.asarray(batch_data)
                    batch_data = []
        if not inf:
            break

In [17]:
texts = open("./labeled_data/raw_data/Jinyong/天龙八部.txt",encoding='gbk',errors='ignore').read()

In [18]:
texts = texts.replace("\u3000","")

In [19]:
valid_gen = data_generator(valid_stories,sample_len=500,batch_size=2,inf=True)

In [20]:
valid_gen.__next__().shape

(2, 500)

In [21]:
text_gen = data_generator_content(texts[528:],sample_len=400,batch_size=2)

In [22]:
text_gen.__next__().shape

(2, 400)

In [23]:
texts[50000:51000]

'，在她手臂上挨挨擦擦，神态极是亲热。那小婢将缰绳交给段誉，道：“这马儿不能鞭打，你待它越好，它跑得越快。”\n段誉道：“是！”心想：“马名黑玫瑰，必是雌马。”说道：“黑玫瑰小姐，小生这厢有礼了！”说着向马作了一揖。那小婢嗤的一笑，道：“你这人倒也有趣。喂，可别摔下来啊。”段誉轻轻跨上马背，向小婢道：“多谢你家小姐！”那小婢笑道：“你不谢我么？”段誉拱手道：“多谢姊姊。回来时我多带些蜜饯果子给你吃。”那小婢道：“果子倒不用带。你千万小心，别骑伤了马儿。”\n来福儿道：“此去一直向北，便是上大理的大路。公子保重。”段誉扬了扬手，那马放开四蹄，几个起落，已在数十丈外。\n这黑玫瑰不用推送，黑夜中奔行如飞，段誉但觉路旁树林犹如倒退一般，不住从眼边跃过，更妙的是马背平稳异常，绝少颠簸起伏，心道：“这马如此快法，明日午后，准能赶到大理。”\n不到一盏茶时分，便已驰出十余里之遥，黑夜中凉风习习，草木清气扑面而来。段誉心道：“良夜驰马，人生一乐。”突然前面有人喝道：“贼贱人，站住！”黑暗中刀光闪动，一柄单刀劈将过来。但黑马奔得极快，这刀砍落时，黑马已纵出丈许之外。段誉回头看去只见两条大汉一持单刀、一持花枪，迈开大步急急赶来。两人破口大骂：“贼贱人！女扮男装，便瞒得过老爷了么？”一幌眼间，黑马已将二人抛得老远。两条大汉虽快步急追，片刻间连叫喊声也听不见了。\n段誉寻思：“这两个莽夫怎地骂我‘贼贱人’，说什么女扮男装？是了，他们要找这黑玫瑰主人的晦气，认马不认人，真是莽撞。”又驰出里许，突然想起：“啊哟，不好！我幸赖马快，逃脱这二人的伏击。瞧这两条大汉似乎武功了得，倘若借马的小姐不知此事，毫没提防的走将出来，难免要遭暗算。我非得回去报讯不可！”当即勒马停步，说道：“黑玫瑰，有人要暗害你家小姐，咱们须得回去告知，请她小心，不可离家外出。”\n当下掉转马头，又从原路回去，将到那大汉先前伏击之处，催马道：“快跑，快跑！”黑玫瑰似解人意，在这两声‘快跑’的催促之下，果然奔驰更快。但那两条大汉却已不知去向。段誉更加急了：“倘若他二人到庄中去袭击那位小姐，岂不糟糕？”他不住吆喝‘快跑’，黑玫瑰四蹄犹如离地一般，疾驰而归。\n将到屋前，忽地两条杆棒贴地挥来，直击马蹄。黑玫瑰不等段誉应变，自行纵跃而过，后腿飞出，砰的一声，将一名持杆棒的汉子踢得直掼了出去。\n黑玫瑰一窜便到门前，黑暗中四五人同时长身

In [ ]:
writer = SummaryWriter('log/finetune')
n_iter = 0
train_loss = 100
val_loss = 100
for epoch in range(100):
    print(f"Epoch {epoch + 1}")
    pb = ProgressBar(10000)
    pb.startjob()
    #for x in data_generator(stories,sample_len=500,batch_size=2):
    for x in data_generator_content(texts[528:],sample_len=500,batch_size=2):
        n_iter += 1
        ret = gpt.train_step(
        (
            tf.constant(x[:,:-1]),
            tf.constant(x[:,1:]))
        )
        writer.add_scalar("train/loss", ret["loss"].numpy(), n_iter)
        train_loss = ret["loss"].numpy()
        if n_iter % 10 == 0:
            valid_x = valid_gen.__next__()
            ret = gpt.eval_step(
            (
                tf.constant(valid_x[:,:-1]),
                tf.constant(valid_x[:,1:]))
            )
            writer.add_scalar("test/loss", ret["loss"].numpy(), n_iter)
            val_loss = ret["loss"].numpy()
        pb.info = f"tl: {train_loss} vl: {val_loss}"
        pb.complete(1)
    print()

Epoch 1
tl: 2.376953125 vl: 1.3564453125 2.44 % [=>-------------------------------------------------] 244/10000 	 used:278s eta:11136 ss

In [25]:
epoch

48

In [ ]:
print(stories[0].to_dungeon_format())

# 使用

In [ ]:
q = f'''你是一个公司老总，你事业正在上升期，你娶了一个美丽的妻子
> 你走进你的家门
'''
ids = cbpe.encode(q)
#print(ids)
#print("+" * 20)
for i in range(100):
    output = gpt(tf.constant([ids]))
    nid = np.argmax(output[0, -1])
    ids += [nid]
    print(i)
    
print(i,cbpe.decode(ids))

In [26]:
gpt.save_weights('./gpt_weight_pretrain/tmp_weight')

# save model (restart kernel here)

In [1]:
%load_ext autoreload
%autoreload 2
import re
import numpy as np
import tensorflow as tf
import os
import sys
from tf2gpt.model import GPT
from utils.story_util import Story,Stories
from utils.progress_bar import ProgressBar
from tensorboardX import SummaryWriter
from tensorflow.keras.utils import multi_gpu_model
import random
tf.keras.backend.set_floatx('float16')

In [2]:
from utils.gpt2_tokenizer import GPT2Tokenizer
cbpe = GPT2Tokenizer(
    'CPM-Generate/bpe_3w_new/vocab.json',
    'CPM-Generate/bpe_3w_new/merges.txt',
    model_file='CPM-Generate/bpe_3w_new/chinese_vocab.model')

In [3]:
#with mirrored_strategy.scope():
gpt = GPT(
    vocab_size=30_000,
    layer_size=32,
    block_size=1024,
    embedding_dropout=0.0,
    embedding_size=2560,
    num_attention_heads=32,
    attention_dropout=0.0,
    residual_dropout=0.0
)
gpt.load_weights('./gpt_weight_pretrain/tmp_weight')

#input_x = tf.keras.layers.Input((499,), dtype=tf.int32)
#outputs = gpt_origin(input_x)

#gpt = tf.keras.Model(inputs=input_x, outputs=outputs)
#gpt = multi_gpu_model(gpt, gpus=8)

print(tf.keras.backend.floatx(), tf.float16, tf.keras.backend.floatx() == tf.float16)
if tf.keras.backend.floatx() == tf.float16:
    for x in gpt.weights:
        assert x.dtype == tf.float16


gpt.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Optimizer
    # Loss function to minimize
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
)

float16 <dtype: 'float16'> True


In [4]:
ids = cbpe.encode('今天天气不错')

for i in range(10):
    output = gpt(tf.constant([ids]))
    nid = np.argmax(output[0, -1])
    ids += [nid]
    print(i, cbpe.decode(ids))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.574 seconds.
Prefix dict has been built successfully.


0 今天天气 不错 
1 今天天气 不错 ,
2 今天天气 不错 , 
3 今天天气 不错 , 咱
4 今天天气 不错 , 咱们
5 今天天气 不错 , 咱们 到
6 今天天气 不错 , 咱们 到 
7 今天天气 不错 , 咱们 到 郊
8 今天天气 不错 , 咱们 到 郊外
9 今天天气 不错 , 咱们 到 郊外 去


In [13]:
def batch_gather(a, b):
    return tf.gather(a, b, batch_dims=1)

def penalize_used(logits, output,penalize):
    # I want to change the indices of logits wherever the index is found in output
    change_tensor = tf.zeros_like(logits, dtype=logits.dtype)
    unique = tf.unique(output[0])[0]
    ones = tf.ones_like(unique, dtype=unique.dtype)
    indices = tf.expand_dims(unique, 1)
    updates = tf.scatter_nd(indices, ones, [logits.shape[1]])
    bool_tensor = tf.expand_dims(tf.cast(updates, tf.bool), 0)
    return tf.compat.v1.where(bool_tensor, logits * penalize, logits)

def top_p_sample(logits, inputs, num_samples=1, p=0.9,k=40,temperature=0.4,penalize=0.4):
    logits = penalize_used(logits,inputs,penalize)
    batch_size, vocab_size = logits.shape
    probs = tf.nn.softmax(logits / temperature, axis=-1)
    # [batch_size, vocab_perm]
    indices = tf.argsort(probs, direction='DESCENDING')
    cumulative_probabilities = tf.math.cumsum(batch_gather(probs, indices), axis=-1, exclusive=False)
    top_k_prob = cumulative_probabilities[0][k]

    # find the top pth index to cut off. careful we don't want to cutoff everything!
    # result will be [batch_size, vocab_perm]
    p_expanded = p if isinstance(p, float) else p#[:, None]
    exclude_mask = tf.logical_not(
        tf.logical_or(
            tf.logical_and(
                cumulative_probabilities < p_expanded, 
                cumulative_probabilities < top_k_prob
            ),
            tf.range(vocab_size)[None] < 1
        )
    )

    # OPTION A - sample in the sorted space, then unsort.
    logits_to_use = batch_gather(logits, indices) - tf.cast(exclude_mask, tf.float16) * 1e4
    sample_perm = tf.random.categorical(logits=logits_to_use, num_samples=num_samples)
    sample = batch_gather(indices, sample_perm)

    return tf.cast(sample, tf.int64)

In [14]:
@tf.function
def serve(inputs):
    return gpt(inputs, kv_cache=None, use_cache=True)


@tf.function
def serve_cache(inputs, kv_cache):
    return gpt(inputs, kv_cache=kv_cache, use_cache=True)

In [15]:
serve_concrete = serve.get_concrete_function(
    tf.TensorSpec(shape=[None, None], dtype=tf.int64, name="inp")
)

layer_size = 32
attention_head = 32
embedding_size = 2560

serve_cache_concrete = serve_cache.get_concrete_function(
    tf.TensorSpec(shape=[None, None], dtype=tf.int64, name="inp"),
    tf.TensorSpec(shape=[
        layer_size, None, 2, attention_head,
        None, embedding_size // attention_head
    ], dtype=tf.float16, name="kv_cache")
)

In [16]:
r = serve_concrete(
    tf.constant([[1]], tf.int64)
)
r2 = serve_cache_concrete(
    tf.constant([[1]], tf.int64),
    r[1]
)

In [17]:
@tf.function
def sample(initial_inputs, length, p,k,temperature,penalize, num_samples=1):
    layer_size = 32
    embedding_size = 2560
    attention_head = 32

    i = tf.constant(0, dtype=tf.int64)
    initial_logits, kv_cache = serve(initial_inputs)
    inputs = top_p_sample(initial_logits[:, -1, :],initial_inputs, p=p,k=k,temperature=temperature,penalize=penalize)
    stores = tf.concat([initial_inputs, inputs], axis=1)

    def _cond(i, inputs, kv_cache, stores):
        return i < length

    def _body(i, inputs, kv_cache, stores):
        new_logits, new_kv_cache = serve_cache(inputs, kv_cache)
        
        new_inputs = top_p_sample(new_logits[:, -1, :],inputs, p=p,k=k,temperature=temperature,penalize=penalize)
        new_stores = tf.concat([stores, new_inputs], axis=-1)
        new_kv_cache = tf.concat([
            kv_cache,
            new_kv_cache
        ], axis=-2)
        new_i = i + 1
        return [new_i, new_inputs, new_kv_cache, new_stores]

    result = tf.while_loop(
        _cond, _body,
        loop_vars=[i, inputs, kv_cache, stores],
        shape_invariants=[
            tf.TensorShape(None),
            tf.TensorShape([None, None]),
            tf.TensorShape([
                layer_size, None, 2,
                attention_head, None,
                embedding_size // attention_head
            ]),
            tf.TensorShape([
                None, None
            ])
        ]
    )
    return result[-1]

In [18]:
gpt.save('./result_weights/tmp_weight', include_optimizer=False, signatures={
    'serving_default': sample.get_concrete_function(
        tf.TensorSpec(shape=[None, None], dtype=tf.int64, name="inp"),
        tf.TensorSpec(shape=[None,], dtype=tf.int64, name="length"),
        tf.TensorSpec(shape=[], dtype=tf.float16, name="p"),
        tf.TensorSpec(shape=[], dtype=tf.int64, name="k"),
        tf.TensorSpec(shape=[], dtype=tf.float16, name="temperature"),
        tf.TensorSpec(shape=[], dtype=tf.float16, name="penalize")
    )
})

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./result_weights/tmp_weight/assets


In [19]:
ids = cbpe.encode('今天天气不错')

ret = sample(
    tf.constant([ids], dtype=tf.int64), # inp
    tf.constant(10, dtype=tf.int64), #length
    tf.constant(0.9, dtype=tf.float16), #p
    tf.constant(40, dtype=tf.int64), #k
    tf.constant(0.4, dtype=tf.float16), # temperature
    tf.constant(0.4, dtype=tf.float16) # penalize
)
print(ret)
print(cbpe.decode(ret.numpy().tolist()[0]))

tf.Tensor(
[[  837   259   497   788     8     9     8 29987    79    81     8  7261
    188    91  3042]], shape=(1, 15), dtype=int64)
今天天气 不错 , 咱们 到 郊外 去 游


In [20]:
!ls -l ./result_weights/tmp_weight

total 23520
drwxr-xr-x 2 tiger tiger     4096 Jul  5 15:21 assets
-rw-r--r-- 1 tiger tiger 24075247 Jul  8 11:57 saved_model.pb
drwxr-xr-x 3 tiger tiger     4096 Jul  8 11:57 variables


# read deploy model (you should restart kernel and go from here)

In [1]:
%load_ext autoreload
%autoreload 2
import re
import numpy as np
import tensorflow as tf
import os
import sys
from tf2gpt.model import GPT
from utils.story_util import Story,Stories
from utils.progress_bar import ProgressBar
from tensorboardX import SummaryWriter
from tensorflow.keras.utils import multi_gpu_model
import random
from utils.gpt2_tokenizer import GPT2Tokenizer
import tensorflow_hub as hub
cbpe = GPT2Tokenizer(
    'CPM-Generate/bpe_3w_new/vocab.json',
    'CPM-Generate/bpe_3w_new/merges.txt',
    model_file='CPM-Generate/bpe_3w_new/chinese_vocab.model')

In [2]:
gpt = hub.load('./result_weights/tmp_weight/')

In [5]:
def sample_gpt(tokenizer, gpt, sentence, number=1, length=20, p=0.9,k=40,temperature=0.4,penalize=0.85):
    inputs = tf.constant([tokenizer.encode(sentence)] * number, dtype=tf.int64)
    length = tf.constant(length, dtype=tf.int64)
    p = tf.constant(p, dtype=tf.float16)
    k = tf.constant(k, dtype=tf.int64)
    temperature = tf.constant(temperature, dtype=tf.float16)
    penalize = tf.constant(penalize, dtype=tf.float16)
    ret = gpt.signatures['serving_default'](
        inp=inputs, 
        length=length, 
        p=p,
        k=k,
        temperature=temperature,
        penalize=penalize)['output_0']
    return [
        tokenizer.decode(s[len(inputs[0]):]).replace(' ', '')
        for s in ret.numpy()
    ]

In [36]:
q = f'''段誉和王语嫣面面相对，呼吸可闻，虽身处污泥，心中却充满了喜乐之情，谁也没想到要爬出井去。两人同时慢慢的伸出手来，四手相握，心意相通。'''
ret = sample_gpt(cbpe, gpt, q, 1, 500, p=0.95,k=8,temperature=1,penalize=0.75)
for x in ret:
    print(x)
    print('-' * 20)

他二人虽然不知道对方此刻身上发生了什么事,但却都是十分欢喜。
王语嫣道:“你身上没受什么伤,为什么我瞧着你,便像是中了‘九阴魔’掌似的?”段誉摇了摇头,道:“这个...段姑娘,你说我中的是‘九阴魔掌’,这‘九阴魔掌’的掌力,是不是也和你身上的伤一样?”
王语嫣心中一凛,颤声道:“那是我中了什么人的毒,以致毒发而中了‘九阴魔掌’?”段誉道:“是啊,那是‘九阴魔掌’的掌力。”他说了自己的掌法和掌法的名字,那是怕王语嫣见到他的掌法之后,立时便会知道他也曾中毒,是以故意说得轻描淡写。王语嫣道:“那是谁的掌法?”段誉道:“你问那是谁的掌法?”
王语嫣登时想起了慕容复,心中一酸,泪水夺眶而出,但随即想到:“段公子也曾中过‘九阴魔掌’的掌力,他的掌法和我师父一模一样,难道是他?可是他身上的毒也太厉害了,我怎能不担心?我是他的‘贤内助’,他怎么会......他...唉!”
段誉道:“王姑娘,我的掌法是师父教的,我不知道。”
王语嫣心中一酸,泪水夺眶而出,但随即想到:“段公子也曾中过‘九阴魔掌’的掌力,他的掌法和
--------------------


In [10]:
class Story():
    def __init__(self,beginning,story_max_len=150,context_len=12):
        self.story = [beginning]
        self.story_max_len = story_max_len
        self.context_len = context_len
    
    def action(self,action):
        action_str = "> 你" + action
        self.story.append(action_str)
        q = "\n".join(self.story[-self.context_len:]) + "\n"
        response = sample_gpt(cbpe, gpt, q, 1, 150)[0]
        
        #responsesp = response.split("。")
        #if(len(responsesp) > 2):
        #    response = ''.join(responsesp[:-1])
        #else:
        #    response = response.split(">")[0]
        #response = "。".join([:-1])
        response = response.split(">")[0].strip()
        #response = "\n".join(response.split("\n")[:-1])
        self.story.append(response)
        
    def interactive(self):
        print("\n".join(self.story))
        while True:
            action = input("> 你")
            self.action(action)
            print(self.story[-1])

In [31]:
begin = f'''你是一家夜总会的经理，在这个城市里，你希望开创自己的一番事业，今天晚上，有一个大客户要光临你的夜总会，你得好好准备'''
story = Story(begin)

In [32]:
story.interactive()

你是一家夜总会的经理，在这个城市里，你希望开创自己的一番事业，今天晚上，有一个大客户要光临你的夜总会，你得好好准备


> 你 走进洗手间


晚上8点,夜总会老板李老二来上班了,他早早来到夜总会,仔细观察起夜总会的环境来。看得出来,他对这里的营业气氛很满意,很大方地给了每个员工这里带领员水果,打扫场地,这里的员工让人很有归属感。


> 你 走进洗手间上厕所


你悄悄走到李老二的身后,看到他开门的时候动作很轻并没有说什么,不过他却疑惑地打量起洗手间来。洗手间很大,很潮湿,一点也不安稳,李老二皱了皱眉头,心里这样想着。


...

<eod


> 你 问警员“你是谁，为什么在这里”


> 你 离开


警察缓缓转过身,指着一位坐在副驾的男子。


KeyboardInterrupt: Interrupted by user